In [ ]:
%matplotlib nbagg
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import interpolate
from scipy import integrate
import matplotlib.pyplot as plt
from matplotlib import cm

from tqdm import tqdm

import pickle

from numba import jit

import linear_operators as lin_ops
import boundary_conditions as bcs
import time_steppers as tstep
import rom_functions as rom

import time as tlib


plt.rcParams.update({"text.usetex":True,\
                     "font.family":"serif",\
                     "font.sans-serif":["Computer Modern"]})

# Jet flow setup

In [ ]:
class jet_class:
    
    def __init__(self,r,z,Re,theta0):
        
        self.r = r
        self.z = z
        self.rowsu = len(r) + 2
        self.colsu = len(z) + 1
        self.szu = self.rowsu*self.colsu
        self.rowsv = len(r) + 1
        self.colsv = len(z) + 2
        self.szv = self.rowsv*self.colsv
        self.rowsp = len(r)
        self.colsp = len(z)
        self.szp = self.rowsp*self.colsp + 1
        
        self.Re = Re
        self.theta0 = theta0
        self.q_sbf = 0

def output_fields(jet,q):
    
    u = np.zeros((jet.rowsp+1,jet.colsp+1))
    v = u.copy()
    w = u.copy()
    
    dr = jet.r[1] - jet.r[0]
    dz = jet.z[1] - jet.z[0]
    
    r = np.zeros(jet.rowsp+1)
    z = np.zeros(jet.colsp+1)
    for j in range (jet.rowsp+1): r[j] = j*dr
    for j in range (jet.colsp+1): z[j] = j*dz
    Z, R = np.meshgrid(z,r)
    
    for i in range (0,jet.rowsp+1):
        for j in range (0,jet.colsp+1):
            
            ku = i*jet.colsu + j
            kv = i*jet.colsv + j + jet.szu
            
            u[i,j] = 0.5*(q[ku] + q[ku+jet.colsu])
            v[i,j] = 0.5*(q[kv] + q[kv+1])
            w[i,j] = (q[kv+1] - q[kv])/dz - (q[ku+jet.colsu] - q[ku])/dr
            w[i,j] = w[i,j]
            
    return u, v, w, Z, R

## Jet flow parameters

In [ ]:
# ---------------------------------------------------
# --------- Assemble grid ---------------------------
# ---------------------------------------------------

Lr = 4              # Length of radial direction
Nr = 200            # Number of grid points in radial direction
dr = Lr/Nr
r = np.zeros(Nr)
for j in range (0,Nr): r[j] = j*dr + dr/2


Lz = 10             # Length of axial direction
Nz = 250            # Number of grid points in axial direction
dz = Lz/Nz
z = np.zeros(Nz)
for j in range (0,Nz): z[j] = j*dz + dz/2


Re = 2000           # Reynolds number
theta0 = 0.025      # non-dimensional thickness of incoming flow 


r0 = 0.5            # Radial location of center of forcing Gaussian blob
z0 = 1.0            # Axial location of center of forcing Gaussian blob
direction = 'radial'# The forcing impulse enters the radial momentum equation (can change to 'axial')
jet = jet_class(r,z,Re,theta0)  
lops = lin_ops.linear_operators_class(jet,r0,z0,direction)

B_FOM = rom.compute_div_free_B_matrix(lops)

# ---------------------------------------------------
# --------- Specify time horizon --------------------
# ---------------------------------------------------
dt = 0.005           # dt for simulation
n = np.intc(1/dt)    # number of time steps per non-dimensional time unit
N = 50*n             # total number of time steps (i.e., tf = dt*N) # 30*n for Re=1000, 50*n for Re=2000
time = dt*np.arange(0,N,1)
nsave = 100          # save data every nsave snapshots

## Load base flow

In [ ]:
# q_sbf = np.loadtxt("steady_state_solution_Re2000_Nr200_Nz250.txt") # Load the stable steady state
q_sbf = np.loadtxt("data/steady_state_solution_Re2000_Nr200_Nz250.txt")
jet.q_sbf = q_sbf

# Load training data

In [ ]:
fname = 'data/traj_data_train.mat'

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

alphas = data['alphas']
x_traj_data = np.ascontiguousarray(data['x_traj_data'])
t_data = data['t_data']
n_traj = len(alphas)
u_traj = np.zeros((n_traj, 1, len(time)))

## Energy along trajectories

In [ ]:
plt.figure()
energy = np.zeros((len(alphas),len(t_data)))
for k in range (len(alphas)):
    for j in range (len(t_data)):
        energy[k,j] = np.dot(x_traj_data[k,:,j],x_traj_data[k,:,j])
        
    plt.plot(t_data, energy[k,],'k')

# plt.savefig('../figures/jet_flow/energy_trajectories.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/energy_trajectories.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

## Plot a flowfield snapshot

In [ ]:
# Add boundary conditions to the flowfield you wanna plot. This flowfield is a perturbation 
# about the base flow q_sbf

t_idx = 60
traj_idx = 0

flowfield = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_traj_data[traj_idx,:,t_idx]))) + q_sbf

# flowfield = q_sbf

#flowfield = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(Phi_cbal[:,-1])))


# u, v, w, Z, R = output_fields(jet,jet.q_sbf)

u, v, w, Z, R = output_fields(jet,flowfield)

field = w
color_map = 'hot_r'

vminval = -np.max(field)*0
vmaxval = np.max(field)

plt.figure()
plt.contourf(Z,R,w,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
ax = plt.gca()
ax.set_aspect(1)
ax.set_ylim([0,1.5])
ax.set_xlabel('z',fontsize=14)
ax.set_ylabel('r',fontsize=14)

m = plt.cm.ScalarMappable(cmap=color_map)
m.set_array(field)
m.set_clim(vminval,vmaxval)

cbar = plt.gcf().colorbar(m,orientation='horizontal',pad=0.2)
cbar.ax.set_xlabel('Azimuthal vorticity',fontsize=14)
cbar.ax.tick_params(labelsize=14)

# Load covariance balancing samples

In [ ]:
L = 40

n_samples = 10
n_traj = 12

fname = 'data/cbal_L{:d}_{:d}trajs_by_{:d}samples.mat'.format(L, n_traj, n_samples)

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

XT = np.ascontiguousarray(data['X'].T)
YT = np.ascontiguousarray(data['Y'].T)
x_YT = np.ascontiguousarray(data['x_Y'].T)

sx = XT.shape[0]
sg = YT.shape[0]

del data

## Covariance balancing

In [ ]:
U_cbal, s_cbal, VT = sp.linalg.svd(YT@XT.T)
V_cbal = VT.T

var_frac_cbal = np.cumsum(np.square(s_cbal))/np.sum(np.square(s_cbal))

In [ ]:
plt.figure()
plt.semilogy(np.arange(len(s_cbal))+1, 1.0 - var_frac_cbal, 'ko')
plt.show()

In [ ]:
r0 = 100

Phi_cbal = XT.T@V_cbal[:,:r0]/np.sqrt(s_cbal[:r0])
Psi_cbal = YT.T@U_cbal[:,:r0]/np.sqrt(s_cbal[:r0])

z_cbal_traj_data = np.einsum('ikl,kj', x_traj_data, Psi_cbal, optimize=True)

## POD

In [ ]:
U_POD, s_POD, VT_POD = np.linalg.svd(XT.T, full_matrices=False, compute_uv=True)
var_frac_POD = np.cumsum(np.square(s_POD))/np.sum(np.square(s_POD))

In [ ]:
plt.figure()
plt.semilogy(np.arange(len(s_POD))+1, 1.0 - var_frac_POD, 'ko')
plt.show()

In [ ]:
r0 = 100

Phi_POD = U_POD[:,:r0]
Psi_POD = U_POD[:,:r0]

z_POD_traj_data = np.einsum('ikl,kj', x_traj_data, Psi_POD, optimize=True)

# Kernel covariance balancing

In [ ]:
sig_krnl = 8.0

@jit(nopython=True, fastmath=True)
def Krnl(x,y):
    return np.exp( -np.square(np.linalg.norm(x - y)) / (2.0*np.square(sig_krnl)) )

@jit(nopython=True, fastmath=True)
def grad_Krnl(x,y):
    return -Krnl(x,y) * (x-y) / np.square(sig_krnl)

@jit(nopython=True, fastmath=True)
def Ginv_Krnl(x,v):
    return np.square(sig_krnl)*v

@jit(nopython=True, fastmath=True)
def H_Krnl(x,y,v):
    c = Krnl(x,y) / np.square(sig_krnl)
    Hv = c * ( v - (x-y)*np.dot(x-y, v) / np.square(sig_krnl) )
    return Hv

@jit(nopython=True, fastmath=True)
def dy_Krnl(x,y,v):
    return Krnl(x,y) * np.dot(x-y, v) / np.square(sig_krnl)

# center about the lifted origin (steady base flow)
xc = np.zeros(XT.shape[1])

## compute time derivative at each sample point

In [ ]:
XdotT = np.zeros(XT.shape)
scl = np.sqrt(sx)
for i in tqdm(range(sx), position=0, leave=True, desc="evaluating time derivatives"):
    u = np.zeros(1)
    x = XT[i,:] * scl
    XdotT[i,:] = rom.evaluate_f(jet,lops,x,u)

XdotT = np.ascontiguousarray(XdotT)

## Compute kernel Hankel matrix

In [ ]:
@jit(nopython=True, parallel=True, fastmath=True)
def kcbal_matrix_row(i):
    gi = YT[i,:]
    xi = x_YT[i,:]
    H_xcY_i = np.dot(gi, Ginv_Krnl(xi, grad_Krnl(xi, xc)))
    
    H_kcbal_i = np.zeros(sx)
    H_Xdot_kcbal_i = np.zeros(sx)
    H_B_kcbal_i = np.zeros(sx)
    
    for j in range(sx):
        xj = XT[j,:] * np.sqrt(sx)
        H_kcbal_i[j] = (np.dot(gi, Ginv_Krnl(xi, grad_Krnl(xi, xj))) - H_xcY_i) / np.sqrt(sx)
        
        xdotj = XdotT[j,:]
        H_Xdot_kcbal_i[j] = np.dot(gi, Ginv_Krnl(xi, H_Krnl(xi, xj, xdotj)))
        H_B_kcbal_i[j] = np.dot(gi, Ginv_Krnl(xi, H_Krnl(xi, xj, B_FOM)))
    
    return H_xcY_i, H_kcbal_i, H_Xdot_kcbal_i, H_B_kcbal_i

In [ ]:
## Kernel Hankel matrix
H_kcbal = np.zeros((sg, sx))
H_xcY = np.zeros(sg)

## lifted derivative kernel matrices
H_Xdot_kcbal = np.zeros((sg, sx))
H_B_kcbal = np.zeros((sg, sx))

# for i in range(Y_samples.shape[1]):
for i in tqdm(range(sg), position=0, leave=True, desc="evaluating kernel Hankel matrix"):
    H_xcY[i], H_kcbal[i,:], H_Xdot_kcbal[i,:], H_B_kcbal[i,:] = kcbal_matrix_row(i)

In [ ]:
## Hankel matrix SVD
U_kcbal, sig_kcbal, VT_kcbal = np.linalg.svd(H_kcbal, full_matrices=False, compute_uv=True)

## kernel embedding of training data
scl_facs = np.sqrt(sx) * np.reshape(np.sqrt(sig_kcbal), (-1,1))
Z_kcbal = scl_facs * VT_kcbal

## kernel embedding of time derivatives
sig_pinv_kcbal = np.zeros(len(sig_kcbal))
sig_pinv_kcbal[sig_kcbal>0] = 1.0/sig_kcbal[sig_kcbal>0]
Zdot_kcbal = np.sqrt(sig_pinv_kcbal).reshape(-1,1) * np.dot(U_kcbal.T, H_Xdot_kcbal)
B_kcbal = np.sqrt(sig_pinv_kcbal).reshape(-1,1) * np.dot(U_kcbal.T, H_B_kcbal)

## arrange along trajectories
z_kcbal_traj_data = np.array([ Z_kcbal[:, i*len(t_data):(i+1)*len(t_data)] for i in range(n_traj)])
zdot_kcbal_traj_data = np.array([ Zdot_kcbal[:, i*len(t_data):(i+1)*len(t_data)] for i in range(n_traj)])
B_kcbal_traj_data = np.array([ B_kcbal[:, i*len(t_data):(i+1)*len(t_data)] for i in range(n_traj)])

## Save kernel Hankel matrix

In [ ]:
fname = 'data/kcbal_L{:d}_{:d}trajs_by_{:d}samples_ksig{:3e}.mat'.format(L, n_traj, n_samples, sig_krnl)
data = {'H_kcbal': H_kcbal, \
        'H_xcY': H_xcY, \
        'H_Xdot_kcbal': H_Xdot_kcbal, \
        'H_B_kcbal': H_B_kcbal, \
        'xc': xc, \
        'sig_krnl': sig_krnl, \
        'U_kcbal': U_kcbal, \
        'sig_kcbal': sig_kcbal, \
        'VT_kcbal': VT_kcbal, \
        'z_kcbal_traj_data': z_kcbal_traj_data, \
        'zdot_kcbal_traj_data': zdot_kcbal_traj_data, \
        'B_kcbal_traj_data': B_kcbal_traj_data}
        

with open(fname, 'wb') as fh:
   pickle.dump(data, fh)

## Load kernel Hankel matrix

In [ ]:
L = 40

n_samples = 10
n_traj = 12

fname = 'data/kcbal_L{:d}_{:d}trajs_by_{:d}samples_ksig{:3e}.mat'.format(L, n_traj, n_samples, sig_krnl)

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

H_kcbal = data['H_kcbal']
H_xcY = data['H_xcY']
H_Xdot_kcbal = data['H_Xdot_kcbal']
H_B_kcbal = data['H_B_kcbal']
xc = data['xc']
U_kcbal = data['U_kcbal']
sig_kcbal = data['sig_kcbal']
VT_kcbal = data['VT_kcbal']
z_kcbal_traj_data = data['z_kcbal_traj_data']
zdot_kcbal_traj_data = data['zdot_kcbal_traj_data']
B_kcbal_traj_data = data['B_kcbal_traj_data']

In [ ]:
var_frac_kcbal = np.cumsum(np.square(sig_kcbal))/np.sum(np.square(sig_kcbal))

plt.figure()
plt.semilogy(np.arange(500)+1, 1.0 - var_frac_kcbal[:500], 'ko')
plt.show()

## Construct encoder

In [ ]:
@jit(nopython=True, parallel=True, fastmath=True)
def kcbal_enc_single(r, x):
    # dimension r
    # x is a 1-dimensional vector
    
    Ur_scaled = U_kcbal[:,:r] / np.sqrt(sig_kcbal[:r])
    
    # compute inner products with x_vecs (a single vector)
    h = np.array([np.dot(YT[i,:], Ginv_Krnl(x_YT[i,:], grad_Krnl(x_YT[i,:], x))) for i in range(sg)])
    
    z = np.dot(h - H_xcY, Ur_scaled)
    
    return z

@jit(nopython=True, parallel=True, fastmath=True)
def kcbal_enc_multiple(r, X):
    # dimension r
    # X has state vectors as columns
    
    Ur_scaled = U_kcbal[:,:r] / np.sqrt(sig_kcbal[:r])
    
    Z = np.zeros((r, X.shape[1]))
    for j in range(X.shape[1]):
        x = X[:,j]
        
        # compute inner products with x_vecs (a single vector)
        h = np.array([np.dot(YT[i,:], Ginv_Krnl(x_YT[i,:], grad_Krnl(x_YT[i,:], x))) for i in range(sg)])
        
        Z[:,j] = np.dot(h - H_xcY, Ur_scaled)
    
    return Z

def kcbal_enc_trajs(r, x_trajs):
    # dimension r
    # x_trajs is an array of shape n_trajs x n_dim x n_t
    
    z_trajs = np.zeros((x_trajs.shape[0], r, x_trajs.shape[2]))
    for k in tqdm(range(x_trajs.shape[0]), position=0, leave=True, desc="encoding trajectories"):
        z_trajs[k,:,:] = kcbal_enc_multiple(r, x_trajs[k,:,:])
    
    return z_trajs

In [ ]:
# validate encoder
i = 10
print(kcbal_enc_single(5, x_traj_data[0,:,i]))
print(z_kcbal_traj_data[0,:5,i])

## visualize embedding

In [ ]:
colors = plt.cm.viridis(np.linspace(0,1,len(t_data)))

zix0 = 0
zix1 = 1
zix2 = 2

fig = plt.figure()
i0 = 0
i1 = len(t_data)
ax = plt.axes(projection='3d')
ax.scatter(z_kcbal_traj_data[:,zix0,i0], z_kcbal_traj_data[:,zix1,i0], z_kcbal_traj_data[:,zix2,i0], c='k', s=10, alpha=1.0)
for k in range(n_traj):
    for i in range(i0, i1-1):
        ax.plot(z_kcbal_traj_data[k,zix0,i:i+2], z_kcbal_traj_data[k,zix1,i:i+2], z_kcbal_traj_data[k,zix2,i:i+2], color=colors[i])

# ax.set_xlabel('z_1')
# ax.set_ylabel('z_2')
# ax.set_zlabel('z_3')

ax.set_xticks([-50,0,50])
ax.set_yticks([-50,0,50])
ax.set_zticks([-50,0,50])

#ax.set_aspect('equal')
ax.view_init(45, -45)

# plt.savefig('../figures/jet_flow/train_trajs_kCoBRAS_embedding.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/train_trajs_kCoBRAS_embedding.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()


In [ ]:
colors = plt.cm.viridis( (alphas-min(alphas))/(max(alphas)-min(alphas)) )

fig = plt.figure()
i0 = 0
i1 = 10
ax = plt.axes(projection='3d')
ax.scatter(z_kcbal_traj_data[:,0,i0], z_kcbal_traj_data[:,1,i0], z_kcbal_traj_data[:,2,i0], c='k', s=10, alpha=1.0)
for k in range(n_traj):
    ax.plot(z_kcbal_traj_data[k,0,i0:i1], z_kcbal_traj_data[k,1,i0:i1], z_kcbal_traj_data[k,2,i0:i1], color=colors[k])

ax.set_xlabel('z_1')
ax.set_ylabel('z_2')
ax.set_zlabel('z_3')
plt.show()

## Validate time derivatives

In [ ]:
zdot_fd = (z_kcbal_traj_data[:,:,2:-1] - z_kcbal_traj_data[:,:,0:-3])/(t_data[2] - t_data[1])
zdot = zdot_kcbal_traj_data[:,:,1:-2]

In [ ]:
i = 0

In [ ]:
plt.figure()
plt.plot(zdot_fd[0,i,:], 'k-')
plt.plot(zdot[0,i,:], 'b-')
plt.title('zdot_{:d}'.format(i))
plt.show()

i = i + 1

# Kernel POD

## Assemble kernel matrix

In [ ]:
@jit(nopython=True, parallel=True, fastmath=True)
def KPOD_matrix_row(i):
    xi = XT[i,:] * np.sqrt(sx)
    
    K_xcX_i = Krnl(xc, xi)
    K_xcXdot_i = dy_Krnl(xc, xi, XdotT[i,:])
    K_xcB_i = dy_Krnl(xc, xi, B_FOM)
    
    K_XX_i = np.zeros(sx)
    K_XX_i[i] = Krnl(xi, xi)
    for j in range(i+1, sx):
        xj = XT[j,:] * np.sqrt(sx)
        K_XX_i[j] = Krnl(xi, xj)
    
    K_XXdot_i = np.zeros(sx)
    K_XB_i = np.zeros(sx)
    for j in range(sx):
        xj = XT[j,:] * np.sqrt(sx)
        K_XXdot_i[j] = dy_Krnl(xi, xj, XdotT[j,:])
        K_XB_i[j] = dy_Krnl(xi, xj, B_FOM)
    
    return K_XX_i, K_xcX_i, K_XXdot_i, K_xcXdot_i, K_XB_i, K_xcB_i

In [ ]:
# kernel matrix
K_XX = np.zeros((sx, sx))
K_xcX = np.zeros(sx)
K_xcxc = Krnl(xc, xc)

# kernel matrix for time derivatives
K_XXdot = np.zeros((sx, sx))
K_xcXdot = np.zeros(sx)

# kernel matrix for forcing
K_XB = np.zeros((sx, sx))
K_xcB = np.zeros(sx)

for i in tqdm(range(sx), position=0, leave=True, desc="assembling kernel matrix"):
    K_XX_i, K_xcX[i], K_XXdot[i,:], K_xcXdot[i], K_XB[i,:], K_xcB[i] = KPOD_matrix_row(i)
    K_XX[i,i:] = K_XX_i[i:]
    K_XX[i:,i] = K_XX_i[i:]

K_KPOD = (K_XX - np.reshape(K_xcX, (-1,1)) - np.reshape(K_xcX, (1,-1)) + K_xcxc) / sx
Kdot_KPOD = (K_XXdot - np.reshape(K_xcXdot, (1,-1))) / np.sqrt(sx)
KB_KPOD = (K_XB - np.reshape(K_xcB, (1,-1))) / np.sqrt(sx)

In [ ]:
ssq, V = sp.linalg.eigh(K_KPOD)
sig_KPOD = np.sqrt(np.maximum(0.0,ssq[::-1]))
VT_KPOD = np.transpose(V[:,::-1])

## kernel embedding of training data
scl_facs = np.sqrt(sx) * np.reshape(sig_KPOD, (-1,1))
Z_KPOD = scl_facs * VT_KPOD

## kernel embedding of time derivatives
sig_pinv_KPOD = np.zeros(len(sig_KPOD))
sig_pinv_KPOD[sig_KPOD>0] = 1.0/sig_KPOD[sig_KPOD>0]
Zdot_KPOD = sig_pinv_KPOD.reshape(-1,1) * np.dot(VT_KPOD, Kdot_KPOD)
B_KPOD = sig_pinv_KPOD.reshape(-1,1) * np.dot(VT_KPOD, KB_KPOD)

## arrange along trajectories
z_KPOD_traj_data = np.array([ Z_KPOD[:, i*len(t_data):(i+1)*len(t_data)] for i in range(n_traj)])
zdot_KPOD_traj_data = np.array([ Zdot_KPOD[:, i*len(t_data):(i+1)*len(t_data)] for i in range(n_traj)])
B_KPOD_traj_data = np.array([ B_KPOD[:, i*len(t_data):(i+1)*len(t_data)] for i in range(n_traj)])

## Save kernel matrix

In [ ]:
fname = 'data/kpod_ksig{:3e}.mat'.format(sig_krnl)
data = {'K_XX': K_XX,\
        'K_xcX': K_xcX, \
        'K_xcxc': K_xcxc, \
        'K_XXdot': K_XXdot, \
        'K_xcXdot': K_xcXdot, \
        'K_XB': K_XB, \
        'K_xcB': K_xcB, \
        'K_KPOD': K_KPOD, \
        'Kdot_KPOD': Kdot_KPOD, \
        'KB_KPOD': KB_KPOD, \
        'xc': xc, \
        'sig_krnl': sig_krnl, \
        'sig_KPOD': sig_KPOD, \
        'VT_KPOD': VT_KPOD, \
        'z_KPOD_traj_data': z_KPOD_traj_data, \
        'zdot_KPOD_traj_data': zdot_KPOD_traj_data, \
        'B_KPOD_traj_data': B_KPOD_traj_data}

with open(fname, 'wb') as fh:
   pickle.dump(data, fh)

## Load kernel matrix

In [ ]:
fname = 'data/kpod_ksig{:3e}.mat'.format(sig_krnl)

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

K_XX = data['K_XX']
K_xcX = data['K_xcX']
K_xcxc = data['K_xcxc']
K_XXdot = data['K_XXdot']
K_xcXdot = data['K_xcXdot']
K_XB = data['K_XB']
K_xcB = data['K_xcB']
K_KPOD = data['K_KPOD']
Kdot_KPOD = data['Kdot_KPOD']
KB_KPOD = data['KB_KPOD']
sig_KPOD= data['sig_KPOD']
VT_KPOD = data['VT_KPOD']
z_KPOD_traj_data = data['z_KPOD_traj_data']
zdot_KPOD_traj_data = data['zdot_KPOD_traj_data']
B_KPOD_traj_data = data['B_KPOD_traj_data']

In [ ]:
var_frac_sig_KPOD = np.cumsum(np.square(sig_KPOD))/np.sum(np.square(sig_KPOD))

plt.figure()
plt.semilogy(np.arange(500)+1, 1.0 - var_frac_sig_KPOD[:500], 'ko')
plt.show()

## KPOD encoder

In [ ]:
@jit(nopython=True, parallel=True, fastmath=True)
def KPOD_enc_single(r, x):
    # dimension r
    # x is a 1-dimensional vector
    
    Vr_scaled = VT_KPOD[:r,:].T / (sig_KPOD[:r] * np.sqrt(len(K_xcX)))
    c = np.sqrt(sx)
    
    # compute inner products with x (a single vector)
    K_xX = np.array([Krnl(x, c*XT[i,:]) for i in range(sx)])
    K_xxc = Krnl(x, xc)
    z = np.dot(K_xX - K_xcX - K_xxc + K_xcxc, Vr_scaled)
    
    return z

@jit(nopython=True, parallel=True, fastmath=True)
def KPOD_enc_multiple(r, X):
    # dimension r
    # X has state vectors as columns
    
    Vr_scaled = VT_KPOD[:r,:].T / (sig_KPOD[:r] * np.sqrt(len(K_xcX)))
    c = np.sqrt(sx)
    
    Z = np.zeros((r, X.shape[1]))
    for j in range(X.shape[1]):
        x = X[:,j]
        # compute inner products with x (a single vector)
        K_xX = np.array([Krnl(x, c*XT[i,:]) for i in range(sx)])
        K_xxc = Krnl(x, xc)
        Z[:,j] = np.dot(K_xX - K_xcX - K_xxc + K_xcxc, Vr_scaled)
    
    return Z

def KPOD_enc_trajs(r, x_trajs):
    # dimension r
    # x_trajs is an array of shape n_trajs x n_dim x n_t
    
    z_trajs = np.zeros((x_trajs.shape[0], r, x_trajs.shape[2]))
    for k in tqdm(range(x_trajs.shape[0]), position=0, leave=True, desc="encoding trajectories"):
        z_trajs[k,:,:] = KPOD_enc_multiple(r, x_trajs[k,:,:])
    
    return z_trajs

In [ ]:
# validate encoder
i = 50
print(KPOD_enc_single(5, x_traj_data[0,:,i]))
print(z_KPOD_traj_data[0,:5,i])

## Visualize KPOD embedding

In [ ]:
colors = plt.cm.viridis(np.linspace(0,1,len(t_data)))

fig = plt.figure()
i0 = 0
i1 = len(t_data)
ax = plt.axes(projection='3d')
ax.scatter(z_KPOD_traj_data[:,0,i0], z_KPOD_traj_data[:,1,i0], z_KPOD_traj_data[:,2,i0], c='k', s=10, alpha=1.0)
for k in range(n_traj):
    for i in range(i0, i1-1):
        ax.plot(z_KPOD_traj_data[k,0,i:i+2], z_KPOD_traj_data[k,1,i:i+2], z_KPOD_traj_data[k,2,i:i+2], color=colors[i])

# ax.set_xlabel('z_1')
# ax.set_ylabel('z_2')
# ax.set_zlabel('z_3')

ax.set_xticks([-0.7,0.0])
ax.set_yticks([-0.5,0,0.5])
ax.set_zticks([-0.5,0,0.5])

#ax.set_aspect('equal')
ax.view_init(30, -60)

# plt.savefig('../figures/jet_flow/train_trajs_KPOD_embedding.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/train_trajs_KPOD_embedding.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
colors = plt.cm.viridis( (alphas-min(alphas))/(max(alphas)-min(alphas)) )

fig = plt.figure()
i0 = 0
i1 = 10
ax = plt.axes(projection='3d')
ax.scatter(z_KPOD_traj_data[:,0,i0], z_KPOD_traj_data[:,1,i0], z_KPOD_traj_data[:,2,i0], c='k', s=10, alpha=1.0)
for k in range(n_traj):
    ax.plot(z_KPOD_traj_data[k,0,i0:i1], z_KPOD_traj_data[k,1,i0:i1], z_KPOD_traj_data[k,2,i0:i1], color=colors[k])

ax.set_xlabel('z_1')
ax.set_ylabel('z_2')
ax.set_zlabel('z_3')
plt.show()

## Validate time derivatives

In [ ]:
zdot_fd = (z_KPOD_traj_data[:,:,2:-1] - z_KPOD_traj_data[:,:,0:-3])/(t_data[2] - t_data[1])
zdot = zdot_KPOD_traj_data[:,:,1:-2]

In [ ]:
i = 0

In [ ]:
plt.figure()
plt.plot(zdot_fd[0,i,:], 'k-')
plt.plot(zdot[0,i,:], 'b-')
plt.title('zdot_{:d}'.format(i))
plt.show()

i = i + 1

# Trajectory interpolation

In [ ]:
@jit(nopython=True, fastmath=True)
def interp1d_with_deriv(t, x, dxdt, t_new):
    A = np.array([[1.0, 0.0, 0.0, 0.0], \
                  [1.0, 1.0, 1.0, 1.0], \
                  [0.0, 1.0, 0.0, 0.0], \
                  [0.0, 1.0, 2.0, 3.0]])
    dt = t[1:] - t[0:-1]
    X = np.zeros((4, len(t)-1))
    X[0,:] = x[0:-1]
    X[1,:] = x[1:]
    X[2,:] = dt*dxdt[0:-1]
    X[3,:] = dt*dxdt[1:]
    
    # coefficients on piecewise cubic interpolant
    # with transformed time tau = (t - t[i])/(t[i+1] - t[i])
    C = np.linalg.solve(A, X)
    
    x_new = np.zeros(len(t_new))
    dxdt_new = np.zeros(len(t_new))
    i = 0
    for j in range(len(t_new)):
        # find the first index i so that t[i] <= t_new[j] <= t[i+1]
        while t[i+1] < t_new[j]:
            i = i+1
        
        # interpolate over the ith interval
        tau = (t_new[j] - t[i]) / (t[i+1] - t[i])
        x_new[j] = C[0,i] + C[1,i]*tau + C[2,i]*tau**2 + C[3,i]*tau**3
        dxdt_new[j] = (C[1,i] + 2*C[2,i]*tau + 3*C[3,i]*tau**2) / (t[i+1] - t[i])
    
    return x_new, dxdt_new

@jit(nopython=True, parallel=True, fastmath=True)
def interp_traj_data(t, x_traj, xdot_traj, t_new):
    nn_traj = x_traj.shape[0]
    rr = x_traj.shape[1]
    x_traj_new = np.zeros((nn_traj, rr, len(t_new)))
    xdot_traj_new = np.zeros((nn_traj, rr, len(t_new)))
    for k in range(nn_traj):
        for i in range(rr):
            x_traj_new[k,i,:], xdot_traj_new[k,i,:] = interp1d_with_deriv(t, x_traj[k,i,:], xdot_traj[k,i,:], t_new)
    
    return x_traj_new, xdot_traj_new


In [ ]:
i = 0
t_data_fine = np.linspace(t_data[0], t_data[-1], 1000)
zi_fine, zidot_fine = interp1d_with_deriv(t_data, z_kcbal_traj_data[0,i,:], zdot_kcbal_traj_data[0,i,:], t_data_fine)

plt.figure()
plt.plot(t_data, z_kcbal_traj_data[0,i,:], 'ko')
plt.plot(t_data_fine, zi_fine, 'b-')
plt.show()

## Construct fine-scale data via interpolation

In [ ]:
t_data_fine = np.linspace(t_data[0], t_data[-1], 1000)

## kernel covariance balancing
# interpolate projected trajectory data
z_cbal_traj_data_fine = sp.interpolate.interp1d(t_data, z_cbal_traj_data, kind='cubic', axis=2)(t_data_fine)
# interpolate embedded trajectories
z_kcbal_traj_data_fine, zdot_kcbal_traj_data_fine = interp_traj_data(t_data, \
                                                                     z_kcbal_traj_data, \
                                                                     zdot_kcbal_traj_data, \
                                                                     t_data_fine)
# interpolate embedded input vector field
B_kcbal_traj_data_fine = sp.interpolate.interp1d(t_data, B_kcbal_traj_data, kind='cubic', axis=2)(t_data_fine)

Z_cbal_fine = np.hstack(z_cbal_traj_data_fine)
Z_kcbal_fine = np.hstack(z_kcbal_traj_data_fine)
Zdot_kcbal_fine = np.hstack(zdot_kcbal_traj_data_fine)
B_kcbal_fine = np.hstack(B_kcbal_traj_data_fine)

## KPOD
# interpolate projected trajectory data
z_POD_traj_data_fine = sp.interpolate.interp1d(t_data, z_POD_traj_data, kind='cubic', axis=2)(t_data_fine)
# interpolate embedded trajectories
z_KPOD_traj_data_fine, zdot_KPOD_traj_data_fine = interp_traj_data(t_data, \
                                                                     z_KPOD_traj_data, \
                                                                     zdot_KPOD_traj_data, \
                                                                     t_data_fine)
# interpolate embedded input vector field
B_KPOD_traj_data_fine = sp.interpolate.interp1d(t_data, B_KPOD_traj_data, kind='cubic', axis=2)(t_data_fine)

Z_POD_fine = np.hstack(z_POD_traj_data_fine)
Z_KPOD_fine = np.hstack(z_KPOD_traj_data_fine)
Zdot_KPOD_fine = np.hstack(zdot_KPOD_traj_data_fine)
B_KPOD_fine = np.hstack(B_KPOD_traj_data_fine)

## Visualize interpolated trajectories

In [ ]:
colors = plt.cm.viridis(np.linspace(0,1,len(t_data)))
colors_fine = plt.cm.viridis(np.linspace(0,1,len(t_data_fine)))

zix0 = 0
zix1 = 1
zix2 = 2

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(z_kcbal_traj_data[:,zix0,0], z_kcbal_traj_data[:,zix1,0], z_kcbal_traj_data[:,zix2,0], \
           c='k', s=10, alpha=1.0)
for k in range(n_traj):
    ax.scatter(z_kcbal_traj_data[k,zix0,1:], z_kcbal_traj_data[k,zix1,1:], z_kcbal_traj_data[k,zix2,1:], \
               c=colors[1:], s=10, alpha=1.0)
    for i in range(len(t_data_fine)-1):
        ax.plot(z_kcbal_traj_data_fine[k,zix0,i:i+2], \
                z_kcbal_traj_data_fine[k,zix1,i:i+2], \
                z_kcbal_traj_data_fine[k,zix2,i:i+2], '-', color=colors_fine[i])

# ax.set_xlabel('z_1')
# ax.set_ylabel('z_2')
# ax.set_zlabel('z_3')

ax.set_xticks([-50,0,50])
ax.set_yticks([-50,0,50])
ax.set_zticks([-50,0,50])

#ax.set_aspect('equal')
# ax.view_init(45, -45)
ax.view_init(30, -30)

# plt.savefig('../figures/jet_flow/train_trajs_kCoBRAS_embedding.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/train_trajs_kCoBRAS_embedding.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
colors = plt.cm.viridis(np.linspace(0,1,len(t_data)))
colors_fine = plt.cm.viridis(np.linspace(0,1,len(t_data_fine)))

zix0 = 0
zix1 = 1
zix2 = 2

fig = plt.figure()
ax = plt.axes(projection='3d')
for k in range(n_traj):
    ax.scatter(z_KPOD_traj_data[k,zix0,1:], z_KPOD_traj_data[k,zix1,1:], z_KPOD_traj_data[k,zix2,1:], \
               c=colors[1:], s=10, alpha=1.0)
    for i in range(len(t_data_fine)-1):
        ax.plot(z_KPOD_traj_data_fine[k,zix0,i:i+2], \
                z_KPOD_traj_data_fine[k,zix1,i:i+2], \
                z_KPOD_traj_data_fine[k,zix2,i:i+2], '-', color=colors_fine[i])

ax.scatter(z_KPOD_traj_data[:,0,i0], z_KPOD_traj_data[:,1,i0], z_KPOD_traj_data[:,2,i0], c='k', s=10, alpha=1.0)

# ax.set_xlabel('z_1')
# ax.set_ylabel('z_2')
# ax.set_zlabel('z_3')

ax.set_xticks([-0.7,0.0])
ax.set_yticks([-0.5,0,0.5])
ax.set_zticks([-0.5,0,0.5])

#ax.set_aspect('equal')
ax.view_init(30, -60)

# plt.savefig('../figures/jet_flow/train_trajs_KPOD_embedding.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/train_trajs_KPOD_embedding.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

# Evaluation data

In [ ]:
fname = 'data/traj_data_eval.mat'

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

alphas_eval = data['alphas']
x_traj_eval = data['x_traj_data']
t_data_eval = data['t_data']
n_traj_eval = len(alphas_eval)
u_traj_eval = np.zeros((n_traj_eval, 1, len(time)))

## Embedding of evaluation data

In [ ]:
## compute latent space embeddings
z_kcbal_eval = kcbal_enc_trajs(500, x_traj_eval)
z_KPOD_eval = KPOD_enc_trajs(500, x_traj_eval)

### Save embedding

In [ ]:
## save latent space embeddings
fname = 'data/traj_data_eval_embedding_ksig{:3e}_tmp.mat'.format(sig_krnl)
data = {'z_kcbal_eval': z_kcbal_eval,\
        'z_KPOD_eval': z_KPOD_eval}

with open(fname, 'wb') as fh:
   pickle.dump(data, fh)

### Load embedding

In [ ]:
## load latent space embeddings
fname = 'data/traj_data_eval_embedding_ksig{:3e}.mat'.format(sig_krnl)

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

z_kcbal_eval = data['z_kcbal_eval']
z_KPOD_eval = data['z_KPOD_eval']

# Testing data

In [ ]:
fname = 'data/traj_data_test.mat'

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

alphas_test = data['alphas']
x_traj_test = data['x_traj_data']
t_data_test = data['t_data']
n_traj_test = len(alphas_test)
u_traj_eval = np.zeros((n_traj_test, 1, len(time)))

## Embedding of testing data

In [ ]:
## compute latent space embeddings
z_kcbal_test = kcbal_enc_trajs(500, x_traj_test)
z_KPOD_test = KPOD_enc_trajs(500, x_traj_test)

### Save embedding

In [ ]:
## save latent space embeddings
fname = 'data/traj_data_test_embedding_ksig{:3e}_tmp.mat'.format(sig_krnl)
data = {'z_kcbal_test': z_kcbal_test,\
        'z_KPOD_test': z_KPOD_test}

with open(fname, 'wb') as fh:
   pickle.dump(data, fh)

### Load embedding

In [ ]:
## load latent space embeddings
fname = 'data/traj_data_test_embedding_ksig{:3e}.mat'.format(sig_krnl)

with open(fname, 'rb') as fh:
   data = pickle.load(fh)

z_kcbal_test = data['z_kcbal_test']
z_KPOD_test = data['z_KPOD_test']

# Fit discrete-time dynamics

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge

In [ ]:
r = 15

## kernel covariance balancing

# data matrices
Z_cbal = np.hstack(z_cbal_traj_data)
Z_kcbal = np.hstack(z_kcbal_traj_data[:,:r,:])
scaler_kcbal = StandardScaler(with_mean=False, with_std=True).fit(Z_kcbal.T)

# data matrices shifted by a single time step
Z0_kcbal = np.hstack(z_kcbal_traj_data[:,:r,0:-2])
Z1_kcbal = np.hstack(z_kcbal_traj_data[:,:r,1:-1])

## kernel POD

# data matrices
Z_POD = np.hstack(z_POD_traj_data)
Z_KPOD = np.hstack(z_KPOD_traj_data[:,:r,:])
scaler_KPOD = StandardScaler(with_mean=False, with_std=True).fit(Z_KPOD.T)

# data matrices shifted by a single time step
Z0_KPOD = np.hstack(z_KPOD_traj_data[:,:r,0:-2])
Z1_KPOD = np.hstack(z_KPOD_traj_data[:,:r,1:-1])

## Reconstruction map

In [ ]:
kr_rec_kcbal = GridSearchCV(KernelRidge(kernel="rbf"),\
                               param_grid={"alpha": np.logspace(-2, 0, 21), \
                                           "gamma": np.logspace(-2, 0, 21)}, n_jobs=-1 )

kr_rec_kcbal.fit(scaler_kcbal.transform(Z_kcbal.T), Z_cbal.T)
print(kr_rec_kcbal.best_params_)
print(kr_rec_kcbal.score(scaler_kcbal.transform(Z_kcbal.T), Z_cbal.T))

In [ ]:
kr_rec_KPOD = GridSearchCV(KernelRidge(kernel="rbf"),\
                               param_grid={"alpha": np.logspace(-3, 1, 21), \
                                           "gamma": np.logspace(-2, 0, 21)}, n_jobs=-1 )

kr_rec_KPOD.fit(scaler_KPOD.transform(Z_KPOD.T), Z_POD.T)
print(kr_rec_KPOD.best_params_)
print(kr_rec_KPOD.score(scaler_KPOD.transform(Z_KPOD.T), Z_POD.T))

In [ ]:
def kcbal_rec_fun(Z):
    if len(Z.shape) == 1:
        xT_r0 = kr_rec_kcbal.predict(scaler_kcbal.transform(Z.reshape(1,-1)))
        X_rec = np.reshape(np.dot(xT_r0, Phi_cbal.T), (-1))
    elif len(Z.shape) == 2:
        XT_r0 = kr_rec_kcbal.predict(scaler_kcbal.transform(Z.T))
        X_rec = np.transpose(np.dot(XT_r0, Phi_cbal.T))
    
    return X_rec

def KPOD_rec_fun(Z):
    if len(Z.shape) == 1:
        xT_r0 = kr_rec_KPOD.predict(scaler_KPOD.transform(Z.reshape(1,-1)))
        X_rec = np.reshape(np.dot(xT_r0, Phi_POD.T), (-1))
    elif len(Z.shape) == 2:
        XT_r0 = kr_rec_KPOD.predict(scaler_KPOD.transform(Z.T))
        X_rec = np.transpose(np.dot(XT_r0, Phi_POD.T))
    
    return X_rec

In [ ]:
t_idx = 30
traj_idx = np.random.randint(n_traj)

flowfield_gt = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_traj_data[traj_idx,:,t_idx]))) + q_sbf
flowfield_kcbal = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(kcbal_rec_fun(z_kcbal_traj_data[traj_idx,:r,t_idx]) ))) + q_sbf
flowfield_KPOD = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(KPOD_rec_fun(z_KPOD_traj_data[traj_idx,:r,t_idx]) ))) + q_sbf

# flowfield_gt = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_traj_eval[traj_idx,:,t_idx]))) + q_sbf
# flowfield_kcbal = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(kcbal_rec_fun(z_kcbal_eval[traj_idx,:r,t_idx]) ))) + q_sbf
# flowfield_KPOD = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(KPOD_rec_fun(z_KPOD_eval[traj_idx,:r,t_idx]) ))) + q_sbf

_, _, w_gt, Z, R = output_fields(jet,flowfield_gt)
_, _, w_kcbal, _, _ = output_fields(jet,flowfield_kcbal)
_, _, w_KPOD, _, _ = output_fields(jet,flowfield_KPOD)


vminval = 0.0
vmaxval = 10.0
color_map = 'hot_r'

fig, axs = plt.subplots(nrows=3, ncols=1, sharex=True)

axs[0].contourf(Z,R,w_gt,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[1].contourf(Z,R,w_kcbal,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[2].contourf(Z,R,w_KPOD,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)

axs[0].set_aspect(1)
axs[1].set_aspect(1)
axs[2].set_aspect(1)

axs[0].set_ylim([0,1.5])
axs[1].set_ylim([0,1.5])
axs[2].set_ylim([0,1.5])

# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(c, cax=cbar_ax)

plt.show()

## Fit dynamics as a map

In [ ]:
kr_dynmap_kcbal = GridSearchCV(KernelRidge(kernel="rbf"), \
                               param_grid={"alpha": np.logspace(-7, -5, 21), \
                                           "gamma": np.logspace(-5, -3, 21)}, n_jobs=-1 )


kr_dynmap_kcbal.fit(scaler_kcbal.transform(Z0_kcbal.T), Z1_kcbal.T)
print(kr_dynmap_kcbal.best_params_)
print(kr_dynmap_kcbal.score(scaler_kcbal.transform(Z0_kcbal.T), Z1_kcbal.T))

In [ ]:
kr_dynmap_KPOD = GridSearchCV(KernelRidge(kernel="rbf"), \
                              param_grid={"alpha": np.logspace(-5, -3, 21), \
                                          "gamma": np.logspace(-4, -2, 21)}, n_jobs=-1 )


kr_dynmap_KPOD.fit(scaler_KPOD.transform(Z0_KPOD.T), Z1_KPOD.T)
print(kr_dynmap_KPOD.best_params_)
print(kr_dynmap_KPOD.score(scaler_KPOD.transform(Z0_KPOD.T), Z1_KPOD.T))

In [ ]:
def kcbal_dynmap_fun(Z):
    if len(Z.shape) == 1:
        zT_pred = kr_dynmap_kcbal.predict(scaler_kcbal.transform(Z.reshape(1,-1)))
        Z_pred = zT_pred.reshape(-1)
    elif len(Z.shape) == 2:
        ZT_pred = kr_dynmap_kcbal.predict(scaler_kcbal.transform(Z.T))
        Z_pred = np.transpose(ZT_pred)
    
    return Z_pred

def KPOD_dynmap_fun(Z):
    if len(Z.shape) == 1:
        zT_pred = kr_dynmap_KPOD.predict(scaler_KPOD.transform(Z.reshape(1,-1)))
        Z_pred = zT_pred.reshape(-1)
    elif len(Z.shape) == 2:
        ZT_pred = kr_dynmap_KPOD.predict(scaler_KPOD.transform(Z.T))
        Z_pred = np.transpose(ZT_pred)
    
    return Z_pred

In [ ]:
## predict trajectory for kernel covariance balancing

n_traj = x_traj_data.shape[0]
z_kcbal_pred = np.inf*np.ones((n_traj, r, len(t_data)))
x_kcbal_pred = np.inf*np.ones(x_traj_data.shape)
for k in tqdm(range(n_traj), position=0, leave=True, desc="simulating trajectories"):
    # simulate the ROM trajectory
    z_kcbal_pred[k,:,0] = z_kcbal_traj_data[k,:r,0] # kcbal_enc(r, x_traj_data[k,:,0])
    x_kcbal_pred[k,:,0] = kcbal_rec_fun(z_kcbal_pred[k,:,0])
    i = 0
    while i < len(t_data)-1 and np.linalg.norm(z_kcbal_pred[k,:,i]) < 1.0e6:
        z_kcbal_pred[k,:,i+1] = kcbal_dynmap_fun(z_kcbal_pred[k,:,i])
        x_kcbal_pred[k,:,i+1] = kcbal_rec_fun(z_kcbal_pred[k,:,i+1])
        i = i+1

## compute error
gt_energy = np.mean(np.square(np.linalg.norm(x_traj_data, axis=1)), axis=1)

kcbal_sq_errors = np.square(np.linalg.norm(x_kcbal_pred - x_traj_data, axis=1)) / \
                    np.reshape(gt_energy, (-1,1))

# compute latent space error
gt_energy_kcbal_latent = np.mean(np.square(np.linalg.norm(z_kcbal_traj_data[:,:r,:], axis=1)), axis=1)

kcbal_sq_errors_latent = np.square(np.linalg.norm(z_kcbal_pred - z_kcbal_traj_data[:,:r,:], axis=1)) / \
                    np.reshape(gt_energy_kcbal_latent, (-1,1))

In [ ]:
## predict trajectory for KPOD

n_traj = x_traj_data.shape[0]
z_KPOD_pred = np.inf*np.ones((n_traj, r, len(t_data)))
x_KPOD_pred = np.inf*np.ones(x_traj_data.shape)
for k in tqdm(range(n_traj), position=0, leave=True, desc="simulating trajectories"):
    # simulate the ROM trajectory
    z_KPOD_pred[k,:,0] = z_KPOD_traj_data[k,:r,0] # KPOD_enc(r, x_traj_data[k,:,0])
    x_KPOD_pred[k,:,0] = KPOD_rec_fun(z_KPOD_pred[k,:,0])
    i = 0
    while i < len(t_data)-1 and np.linalg.norm(z_KPOD_pred[k,:,i]) < 1.0e6:
        z_KPOD_pred[k,:,i+1] = KPOD_dynmap_fun(z_KPOD_pred[k,:,i])
        x_KPOD_pred[k,:,i+1] = KPOD_rec_fun(z_KPOD_pred[k,:,i+1])
        i = i+1

## compute error
gt_energy = np.mean(np.square(np.linalg.norm(x_traj_data, axis=1)), axis=1)

KPOD_sq_errors = np.square(np.linalg.norm(x_KPOD_pred - x_traj_data, axis=1)) / \
                    np.reshape(gt_energy, (-1,1))

# compute latent space error
gt_energy_KPOD_latent = np.mean(np.square(np.linalg.norm(z_KPOD_traj_data[:,:r,:], axis=1)), axis=1)

KPOD_sq_errors_latent = np.square(np.linalg.norm(z_KPOD_pred - z_KPOD_traj_data[:,:r,:], axis=1)) / \
                    np.reshape(gt_energy_KPOD_latent, (-1,1))

In [ ]:
i=0
k=0

plt.figure()
ax = plt.axes(projection='3d')
ax.plot(z_kcbal_traj_data[k,i,:], z_kcbal_traj_data[k,i+1,:], z_kcbal_traj_data[k,i+2,:], 'k-')
ax.plot(z_kcbal_pred[k,i+0,:], z_kcbal_pred[k,i+1,:], z_kcbal_pred[k,i+2,:], 'b--')
plt.show()

plt.figure()
ax = plt.axes(projection='3d')
ax.plot(z_KPOD_traj_data[k,i,:], z_KPOD_traj_data[k,i+1,:], z_KPOD_traj_data[k,i+2,:], 'k-')
ax.plot(z_KPOD_pred[k,i+0,:], z_KPOD_pred[k,i+1,:], z_KPOD_pred[k,i+2,:], 'b--')
plt.show()

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(1,1,1)

k=0
ax1.semilogy(t_data, kcbal_sq_errors[k,:], color='#1f78b4', linestyle='-', label='k-CoBRAS')
ax1.semilogy(t_data, KPOD_sq_errors[k,:], color='#b2df8a', linestyle='--', label='KPOD')
for k in range(1, n_traj):
    ax1.semilogy(t_data, kcbal_sq_errors[k,:], color='#1f78b4', linestyle='-')
    ax1.semilogy(t_data, KPOD_sq_errors[k,:], color='#b2df8a', linestyle='--')

ax1.set_ylabel('normalized square error')

ax1.set_xlabel('t')

#axs[1].set_yticks([])

# ax1.set_xticks(np.arange(6))

ax1.set_ylim([1.0e-5, 1.0e2])

ax1.legend(loc='upper right')

# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True)

## reconstructed error
k=0
axs[0].semilogy(t_data, kcbal_sq_errors[k,:], color='#1f78b4', linestyle='-', label='k-CoBRAS')
axs[0].semilogy(t_data, KPOD_sq_errors[k,:], color='#b2df8a', linestyle='--', label='KPOD')
for k in range(1, n_traj):
    axs[0].semilogy(t_data, kcbal_sq_errors[k,:], color='#1f78b4', linestyle='-')
    axs[0].semilogy(t_data, KPOD_sq_errors[k,:], color='#b2df8a', linestyle='--')


# axs[0].set_xticks([])

# ax1.set_xticks(np.arange(6))

axs[0].set_ylim([1.0e-3, 1.0e1])

axs[0].legend(loc='upper right')

## latent space errors
k=0
axs[1].semilogy(t_data, kcbal_sq_errors_latent[k,:], color='#1f78b4', linestyle='-', label='k-CoBRAS')
axs[1].semilogy(t_data, KPOD_sq_errors_latent[k,:], color='#b2df8a', linestyle='--', label='KPOD')
for k in range(1, n_traj):
    axs[1].semilogy(t_data, kcbal_sq_errors_latent[k,:], color='#1f78b4', linestyle='-')
    axs[1].semilogy(t_data, KPOD_sq_errors_latent[k,:], color='#b2df8a', linestyle='--')


# ax2.set_xticks(np.arange(6))

axs[1].set_ylim([1.0e-5, 1.0e1])


# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('figures/toy_model/test_data_normalized_sqaure_error.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
t_idx = 40
traj_idx = np.argmax(gt_energy)
# traj_idx = np.argsort(gt_energy)[n_traj//2]
# traj_idx = np.random.randint(n_traj)

flowfield_gt = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_traj_data[traj_idx,:,t_idx]))) + q_sbf
flowfield_kcbal = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_kcbal_pred[traj_idx,:,t_idx]))) + q_sbf
flowfield_KPOD = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_KPOD_pred[traj_idx,:,t_idx]))) + q_sbf

_, _, w_gt, Z, R = output_fields(jet,flowfield_gt)
_, _, w_kcbal, _, _ = output_fields(jet,flowfield_kcbal)
_, _, w_KPOD, _, _ = output_fields(jet,flowfield_KPOD)


vminval = 0.0
vmaxval = 10.0
color_map = 'hot_r'

fig, axs = plt.subplots(nrows=3, ncols=1, sharex=True)

axs[0].contourf(Z,R,w_gt,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[1].contourf(Z,R,w_kcbal,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[2].contourf(Z,R,w_KPOD,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)

axs[0].set_aspect(1)
axs[1].set_aspect(1)
axs[2].set_aspect(1)

axs[0].set_ylim([0,1.5])
axs[1].set_ylim([0,1.5])
axs[2].set_ylim([0,1.5])

# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(c, cax=cbar_ax)

plt.show()

## Evaluation data performance

### Predict dynamics

In [ ]:
## predict trajectory for kernel covariance balancing

z_kcbal_pred = np.inf*np.ones((n_traj_eval, r, len(t_data_eval)))
x_kcbal_pred = np.inf*np.ones(x_traj_eval.shape)

for k in tqdm(range(n_traj_eval), position=0, leave=True, desc="simulating trajectories"):
    # simulate the ROM trajectory
    z_kcbal_pred[k,:,0] = z_kcbal_eval[k,:r,0]# kcbal_enc_single(r, x_traj_eval[k,:,0])
    x_kcbal_pred[k,:,0] = kcbal_rec_fun(z_kcbal_pred[k,:,0])
    i = 0
    while i < len(t_data_eval)-1 and np.linalg.norm(z_kcbal_pred[k,:,i]) < 1.0e6:
        z_kcbal_pred[k,:,i+1] = kcbal_dynmap_fun(z_kcbal_pred[k,:,i])
        x_kcbal_pred[k,:,i+1] = kcbal_rec_fun(z_kcbal_pred[k,:,i+1])
        i = i+1

## compute error
gt_energy = np.mean(np.square(np.linalg.norm(x_traj_eval, axis=1)), axis=1)

kcbal_sq_errors = np.square(np.linalg.norm(x_kcbal_pred - x_traj_eval, axis=1)) / \
                    np.reshape(gt_energy, (-1,1))

## compute latent space error
gt_energy_kcbal_latent = np.mean(np.square(np.linalg.norm(z_kcbal_eval[:,:r,:], axis=1)), axis=1)

kcbal_sq_errors_latent = np.square(np.linalg.norm(z_kcbal_pred - z_kcbal_eval[:,:r,:], axis=1)) / \
                    np.reshape(gt_energy_kcbal_latent, (-1,1))

In [ ]:
## predict trajectory for KPOD

z_KPOD_pred = np.inf*np.ones((n_traj_eval, r, len(t_data_eval)))
x_KPOD_pred = np.inf*np.ones(x_traj_eval.shape)

for k in tqdm(range(n_traj_eval), position=0, leave=True, desc="simulating trajectories"):
    # simulate the ROM trajectory
    z_KPOD_pred[k,:,0] = z_KPOD_eval[k,:r,0]# KPOD_enc_single(r, x_traj_eval[k,:,0])
    x_KPOD_pred[k,:,0] = KPOD_rec_fun(z_KPOD_pred[k,:,0])
    i = 0
    while i < len(t_data_eval)-1 and np.linalg.norm(z_KPOD_pred[k,:,i]) < 1.0e6:
        z_KPOD_pred[k,:,i+1] = KPOD_dynmap_fun(z_KPOD_pred[k,:,i])
        x_KPOD_pred[k,:,i+1] = KPOD_rec_fun(z_KPOD_pred[k,:,i+1])
        i = i+1

## compute error
gt_energy = np.mean(np.square(np.linalg.norm(x_traj_eval, axis=1)), axis=1)

KPOD_sq_errors = np.square(np.linalg.norm(x_KPOD_pred - x_traj_eval, axis=1)) / \
                    np.reshape(gt_energy, (-1,1))

## compute latent space error
gt_energy_KPOD_latent = np.mean(np.square(np.linalg.norm(z_KPOD_eval[:,:r,:], axis=1)), axis=1)

KPOD_sq_errors_latent = np.square(np.linalg.norm(z_KPOD_pred - z_KPOD_eval[:,:r,:], axis=1)) / \
                    np.reshape(gt_energy_KPOD_latent, (-1,1))

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True)

## reconstructed error
k=0
axs[0].semilogy(t_data, kcbal_sq_errors[k,:], color='#1f78b4', linestyle='-', label='K-CoBRAS')
axs[0].semilogy(t_data, KPOD_sq_errors[k,:], color='#b2df8a', linestyle='--', label='KPOD')
for k in range(1, n_traj_eval):
    axs[0].semilogy(t_data, kcbal_sq_errors[k,:], color='#1f78b4', linestyle='-')
    axs[0].semilogy(t_data, KPOD_sq_errors[k,:], color='#b2df8a', linestyle='--')


# axs[0].set_xticks([])

# ax1.set_xticks(np.arange(6))

axs[0].set_ylim([1.0e-3, 1.0e1])

axs[0].legend(loc='upper right')

## latent space errors
k=0
axs[1].semilogy(t_data, kcbal_sq_errors_latent[k,:], color='#1f78b4', linestyle='-', label='K-CoBRAS')
axs[1].semilogy(t_data, KPOD_sq_errors_latent[k,:], color='#b2df8a', linestyle='--', label='KPOD')
for k in range(1, n_traj_eval):
    axs[1].semilogy(t_data, kcbal_sq_errors_latent[k,:], color='#1f78b4', linestyle='-')
    axs[1].semilogy(t_data, KPOD_sq_errors_latent[k,:], color='#b2df8a', linestyle='--')


# ax2.set_xticks(np.arange(6))

axs[1].set_ylim([1.0e-5, 1.0e1])


# plt.savefig('../figures/jet_flow/test_data_kernel_GPR_normalized_square_error.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/test_data_kernel_GPR_normalized_square_error.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(1,1,1)

k=0
ax1.semilogy(t_data, kcbal_sq_errors[k,:], color='#1f78b4', linestyle='-', label='K-CoBRAS')
ax1.semilogy(t_data, KPOD_sq_errors[k,:], color='#b2df8a', linestyle='--', label='KPCA')
for k in range(1, n_traj_eval):
    ax1.semilogy(t_data, kcbal_sq_errors[k,:], color='#1f78b4', linestyle='-')
    ax1.semilogy(t_data, KPOD_sq_errors[k,:], color='#b2df8a', linestyle='--')

# ax1.set_ylabel('normalized square error')

# ax1.set_xlabel('t')

#axs[1].set_yticks([])

# ax1.set_xticks(np.arange(6))

ax1.set_ylim([1.0e-4, 1.0e1])

ax1.legend(loc='lower right')

# plt.savefig('../figures/jet_flow/test_data_kernel_KR_normalized_square_error.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/test_data_kernel_KR_normalized_square_error.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()


fig = plt.figure()

ax1 = fig.add_subplot(1,1,1)

k=0
ax1.semilogy(t_data, kcbal_sq_errors_latent[k,:], color='#1f78b4', linestyle='-', label='K-CoBRAS')
ax1.semilogy(t_data, KPOD_sq_errors_latent[k,:], color='#b2df8a', linestyle='--', label='KPCA')
for k in range(1, n_traj_eval):
    ax1.semilogy(t_data, kcbal_sq_errors_latent[k,:], color='#1f78b4', linestyle='-')
    ax1.semilogy(t_data, KPOD_sq_errors_latent[k,:], color='#b2df8a', linestyle='--')

# ax1.set_ylabel('normalized square error')

# ax1.set_xlabel('t')

#axs[1].set_yticks([])

# ax1.set_xticks(np.arange(6))

ax1.set_ylim([1.0e-4, 1.0e1])

ax1.legend(loc='upper right')

# plt.savefig('../figures/jet_flow/test_data_kernel_KR_normalized_square_error_latent.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/test_data_kernel_KR_normalized_square_error_latent.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
t_idx = 10
traj_idx = np.argmax(gt_energy)
# traj_idx = np.argsort(gt_energy)[n_traj//2]
# traj_idx = np.random.randint(n_traj_eval)

print(alphas_eval[traj_idx])

flowfield_gt = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_traj_eval[traj_idx,:,t_idx]))) + q_sbf
flowfield_kcbal = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_kcbal_pred[traj_idx,:,t_idx]))) + q_sbf
flowfield_KPOD = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_KPOD_pred[traj_idx,:,t_idx]))) + q_sbf

_, _, w_gt, Z, R = output_fields(jet,flowfield_gt)
_, _, w_kcbal, _, _ = output_fields(jet,flowfield_kcbal)
_, _, w_KPOD, _, _ = output_fields(jet,flowfield_KPOD)


vminval = 0.0
vmaxval = 10.0
color_map = 'hot_r'

fig, axs = plt.subplots(nrows=3, ncols=1, sharex=True)

axs[0].contourf(Z,R,w_gt,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[1].contourf(Z,R,w_kcbal,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[2].contourf(Z,R,w_KPOD,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)

axs[0].set_aspect(1)
axs[1].set_aspect(1)
axs[2].set_aspect(1)

axs[0].set_ylim([0,1.5])
axs[1].set_ylim([0,1.5])
axs[2].set_ylim([0,1.5])

axs[0].set_title('t = {:.2f}, alpha = {:.4f}'.format(t_data_eval[t_idx], alphas_eval[traj_idx]))

# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(c, cax=cbar_ax)

# plt.savefig('../figures/jet_flow/snapshot_kernel_KR_preds_t5_Emax.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/snapshot_kernel_KR_preds_t5_Emax.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
# k = np.argmax(alphas_eval)
traj_idx = np.argmax(gt_energy)

plt.figure()
ax = plt.axes(projection='3d')
ax.plot(z_kcbal_eval[k,0,:], z_kcbal_eval[k,1,:], z_kcbal_eval[k,2,:], 'k-')
ax.plot(z_kcbal_pred[k,0,:], z_kcbal_pred[k,1,:], z_kcbal_pred[k,2,:], color='b', linestyle='--')

ax.set_xticks([-50,0,50])
ax.set_yticks([-50,0,50])
ax.set_zticks([-50,0,50])

#ax.set_aspect('equal')
# ax.view_init(45, -45)
ax.view_init(30, -30)

# plt.savefig('../figures/jet_flow/kCoBRAS_GPR_latent_pred_Emax.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/kCoBRAS_GPR_latent_pred_Emax.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

plt.figure()
ax = plt.axes(projection='3d')
ax.plot(z_KPOD_eval[k,0,:], z_KPOD_eval[k,1,:], z_KPOD_eval[k,2,:], 'k-')
ax.plot(z_KPOD_pred[k,0,:], z_KPOD_pred[k,1,:], z_KPOD_pred[k,2,:], color='b', linestyle='--')

ax.set_xticks([-0.7,0.0])
ax.set_yticks([-0.5,0,0.5])
ax.set_zticks([-0.5,0,0.5])

#ax.set_aspect('equal')
ax.view_init(30, -60)

# plt.savefig('../figures/jet_flow/KPOD_KR_latent_pred_Emax.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/KPOD_KR_latent_pred_Emax.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

## Testing data performance

In [ ]:
## predict trajectory for kernel covariance balancing

z_kcbal_pred = np.inf*np.ones((n_traj_test, r, len(t_data_test)))
x_kcbal_pred = np.inf*np.ones(x_traj_test.shape)

for k in tqdm(range(n_traj_test), position=0, leave=True, desc="simulating trajectories"):
    # simulate the ROM trajectory
    z_kcbal_pred[k,:,0] = z_kcbal_test[k,:r,0]
    x_kcbal_pred[k,:,0] = kcbal_rec_fun(z_kcbal_pred[k,:,0])
    i = 0
    while i < len(t_data_test)-1 and np.linalg.norm(z_kcbal_pred[k,:,i]) < 1.0e6:
        z_kcbal_pred[k,:,i+1] = kcbal_dynmap_fun(z_kcbal_pred[k,:,i])
        x_kcbal_pred[k,:,i+1] = kcbal_rec_fun(z_kcbal_pred[k,:,i+1])
        i = i+1

## compute error
gt_energy = np.mean(np.square(np.linalg.norm(x_traj_test, axis=1)), axis=1)

kcbal_sq_errors = np.square(np.linalg.norm(x_kcbal_pred - x_traj_test, axis=1)) / \
                    np.reshape(gt_energy, (-1,1))

## compute latent space error
gt_energy_kcbal_latent = np.mean(np.square(np.linalg.norm(z_kcbal_test[:,:r,:], axis=1)), axis=1)

kcbal_sq_errors_latent = np.square(np.linalg.norm(z_kcbal_pred - z_kcbal_test[:,:r,:], axis=1)) / \
                    np.reshape(gt_energy_kcbal_latent, (-1,1))

In [ ]:
## predict trajectory for KPOD

z_KPOD_pred = np.inf*np.ones((n_traj_test, r, len(t_data_test)))
x_KPOD_pred = np.inf*np.ones(x_traj_test.shape)

for k in tqdm(range(n_traj_test), position=0, leave=True, desc="simulating trajectories"):
    # simulate the ROM trajectory
    z_KPOD_pred[k,:,0] = z_KPOD_test[k,:r,0]
    x_KPOD_pred[k,:,0] = KPOD_rec_fun(z_KPOD_pred[k,:,0])
    i = 0
    while i < len(t_data_test)-1 and np.linalg.norm(z_KPOD_pred[k,:,i]) < 1.0e6:
        z_KPOD_pred[k,:,i+1] = KPOD_dynmap_fun(z_KPOD_pred[k,:,i])
        x_KPOD_pred[k,:,i+1] = KPOD_rec_fun(z_KPOD_pred[k,:,i+1])
        i = i+1

## compute error
gt_energy = np.mean(np.square(np.linalg.norm(x_traj_test, axis=1)), axis=1)

KPOD_sq_errors = np.square(np.linalg.norm(x_KPOD_pred - x_traj_test, axis=1)) / \
                    np.reshape(gt_energy, (-1,1))

## compute latent space error
gt_energy_KPOD_latent = np.mean(np.square(np.linalg.norm(z_KPOD_test[:,:r,:], axis=1)), axis=1)

KPOD_sq_errors_latent = np.square(np.linalg.norm(z_KPOD_pred - z_KPOD_test[:,:r,:], axis=1)) / \
                    np.reshape(gt_energy_KPOD_latent, (-1,1))

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(1,1,1)

k=0
ax1.semilogy(t_data, kcbal_sq_errors[k,:], color='#1f78b4', linestyle='-', label='K-CoBRAS')
ax1.semilogy(t_data, KPOD_sq_errors[k,:], color='#b2df8a', linestyle='--', label='KPCA')
for k in range(1, n_traj_test):
    ax1.semilogy(t_data, kcbal_sq_errors[k,:], color='#1f78b4', linestyle='-')
    ax1.semilogy(t_data, KPOD_sq_errors[k,:], color='#b2df8a', linestyle='--')

# ax1.set_ylabel('normalized square error')

# ax1.set_xlabel('t')

#axs[1].set_yticks([])

# ax1.set_xticks(np.arange(6))

ax1.set_ylim([1.0e-3, 1.0e2])

ax1.legend(loc='upper right')

# plt.savefig('../figures/jet_flow/test_data_kernel_KR_normalized_square_error.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/test_data_kernel_KR_normalized_square_error.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()


fig = plt.figure()

ax1 = fig.add_subplot(1,1,1)

k=0
ax1.semilogy(t_data, kcbal_sq_errors_latent[k,:], color='#1f78b4', linestyle='-', label='K-CoBRAS')
ax1.semilogy(t_data, KPOD_sq_errors_latent[k,:], color='#b2df8a', linestyle='--', label='KPCA')
for k in range(1, n_traj_test):
    ax1.semilogy(t_data, kcbal_sq_errors_latent[k,:], color='#1f78b4', linestyle='-')
    ax1.semilogy(t_data, KPOD_sq_errors_latent[k,:], color='#b2df8a', linestyle='--')

# ax1.set_ylabel('normalized square error')

# ax1.set_xlabel('t')

#axs[1].set_yticks([])

# ax1.set_xticks(np.arange(6))

ax1.set_ylim([1.0e-4, 1.0e2])

ax1.legend(loc='upper right')

# plt.savefig('../figures/jet_flow/test_data_kernel_KR_normalized_square_error_latent.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/test_data_kernel_KR_normalized_square_error_latent.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()

In [ ]:
t_idx = 10
traj_idx = np.argmax(gt_energy)
# traj_idx = np.argsort(gt_energy)[n_traj//2]
# traj_idx = np.random.randint(n_traj_test)

print(alphas_test[traj_idx])

flowfield_gt = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_traj_test[traj_idx,:,t_idx]))) + q_sbf
flowfield_kcbal = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_kcbal_pred[traj_idx,:,t_idx]))) + q_sbf
flowfield_KPOD = lops.BC_op.dot(lops.Wsqrtinv.dot(lops.Mapp.dot(x_KPOD_pred[traj_idx,:,t_idx]))) + q_sbf

_, _, w_gt, Z, R = output_fields(jet,flowfield_gt)
_, _, w_kcbal, _, _ = output_fields(jet,flowfield_kcbal)
_, _, w_KPOD, _, _ = output_fields(jet,flowfield_KPOD)


vminval = 0.0
vmaxval = 10.0
color_map = 'hot_r'

fig, axs = plt.subplots(nrows=3, ncols=1, sharex=True)

axs[0].contourf(Z,R,w_gt,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[1].contourf(Z,R,w_kcbal,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)
axs[2].contourf(Z,R,w_KPOD,levels=200,cmap=color_map,vmin=vminval,vmax=vmaxval)

axs[0].set_aspect(1)
axs[1].set_aspect(1)
axs[2].set_aspect(1)

axs[0].set_ylim([0,1.5])
axs[1].set_ylim([0,1.5])
axs[2].set_ylim([0,1.5])

axs[0].set_title('t = {:.2f}, alpha = {:.4f}'.format(t_data_test[t_idx], alphas_test[traj_idx]))

# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(c, cax=cbar_ax)

# plt.savefig('../figures/jet_flow/snapshot_kernel_KR_preds_t5_Emed.png', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)
# plt.savefig('../figures/jet_flow/snapshot_kernel_KR_preds_t5_Emed.eps', dpi=None, facecolor='w', edgecolor='w', \
#             transparent=False, bbox_inches=None, pad_inches=0.0)

plt.show()